In [1]:
%%capture
!pip install timm

In [2]:
import time
from tqdm import tqdm_notebook as tqdm
#import tqdm.notebook import tqdm
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
import timm
import warnings
warnings.filterwarnings('ignore')

In [3]:
!ls -l ../input/midsw251birds2021

In [4]:
timm_models_list = timm.list_models('*efficientnet*')
# timm_models_list

In [5]:
class args:
    lr = 0.00009
    epochs = 30
    batch_size = 16
    num_workers = 8
    folds = 15
    compeition_name = 'midsw251birds2021'

In [6]:
alldf = pd.read_csv(f'../input/{args.compeition_name}/train.csv')
alldf['filename'] = 'train/' + alldf['filename']
# Split the training dataset into a training and a validation
valdf = alldf[::args.folds]
trndf = alldf[~alldf.filename.isin(valdf.filename)]
# Load our test data
tstdf = pd.read_csv(f'../input/{args.compeition_name}/test.csv')
tstdf['filename'] = 'test/' + tstdf['filename']
metadf = pd.read_csv(f'../input/{args.compeition_name}/metadata.csv')
metadf = metadf.set_index('label')
print(f'File shapes -- train : {trndf.shape}, valid : {valdf.shape}, test : {tstdf.shape}')
trndf.head()

In [7]:
imgnetmeans = [0.22363983, 0.18190407, 0.2523437 ]
imgnetstds = [0.32451536, 0.2956294,  0.31335256]
# Using albumentations, check some examples here : https://albumentations.readthedocs.io/en/latest/examples.html 
def trntransforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Transpose(p=0.5),
        A.GaussianBlur(p=0.5),
        A.RandomBrightnessContrast(),
        A.VerticalFlip(),
        ToTensorV2(),
        ])

def tsttransforms():
    return A.Compose([
        ToTensorV2(),
    ])

class BirdDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        self.data = df
        self.img_dir = f'../input/{args.compeition_name}/'
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        fname = self.data.iloc[idx]['filename']
        img_path = f'{self.img_dir}/{fname}'
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image = image)['image']
        image = image.float() / 255.
        label = -1 if self.mode=='test' else self.data.iloc[idx]['label']
        
        return image, label

In [8]:
# Define our dataset
trndataset = BirdDataset(trndf, 'train', trntransforms())
valdataset = BirdDataset(valdf, 'valid', tsttransforms())
tstdataset = BirdDataset(tstdf, 'test', tsttransforms())

In [9]:
# Test the dataset
img, label = next(iter(trndataset))
species = metadf.loc[label]['name']
print(f'Species : {species}')
#Image.fromarray(img)
imgviz = (img * 255).transpose(0, 2).numpy().astype(np.uint8)
Image.fromarray(imgviz)

In [10]:
loaderargs = {'num_workers' : args.num_workers, 'batch_size':args.batch_size, 'pin_memory': False, 'drop_last': False}
trnloader = DataLoader(trndataset, shuffle = True, **loaderargs)
valloader = DataLoader(valdataset, shuffle = False, **loaderargs)
tstloader = DataLoader(tstdataset, shuffle = False, **loaderargs)

In [11]:
# creates efficientnet-b0 architecture
device = torch.device("cuda:0")
# model = timm.create_model('efficientnet_b2', pretrained = True)
# model = timm.create_model('resnet50', pretrained = True)
model = timm.create_model('efficientnet_b3', pretrained = True)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr)
# Loss function
criterion = torch.nn.CrossEntropyLoss()
scheduler = lr_scheduler.StepLR(optimizer, step_size=10)
num_epochs = args.epochs


# from timm.scheduler.cosine_lr import CosineLRScheduler

# n_epochs = num_epochs
# n_warmup_epochs = 2
# n_steps = len(trnloader)

# scheduler = CosineLRScheduler(
#             optimizer,
#             t_initial= n_steps * n_epochs + 1,
#             lr_min=0.01,
#             warmup_lr_init=0.0005,
#             warmup_t= n_steps * n_warmup_epochs + 1)


def mixup_loss(criterion, output, labels_a, labels_b, lam):
    # Partial loss against original labels - partial loss against mixup labels
    # Assigning fractional label based on the amount of space an image has in resulting image
    loss = criterion(output, labels_a) * lam + criterion(output, labels_b) * (1.0 - lam)
    
    return loss

import torch.nn.functional as F
from torch.nn.modules.loss import _WeightedLoss

class LabelSmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth_one_hot(targets: torch.Tensor, n_classes: int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = torch.empty(size=(targets.size(0), n_classes),
                                  device=targets.device) \
                .fill_(smoothing / (n_classes - 1)) \
                .scatter_(1, targets.data.unsqueeze(1), 1. - smoothing)
        return targets

    def forward(self, inputs, targets):
        targets = LabelSmoothCrossEntropyLoss._smooth_one_hot(targets, inputs.size(-1),
                                                              self.smoothing)
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()

        return loss
criterion = LabelSmoothCrossEntropyLoss(smoothing=0.1)

In [12]:
# Base code
since = time.time()
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
#     scheduler.step()
    model.train()
    running_loss = 0.0
    tk0 = tqdm(trnloader, total=int(len(trnloader)))
    for step, batch in enumerate(tk0):
        inputs = batch[0].to(device, dtype=torch.float)
        labels = batch[1].to(device).long()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        tk0.set_postfix(train_loss=(running_loss / (step+1)))
    
    scheduler.step(epoch=epoch)    
    
    valpreds = []
    model.eval()
    running_loss = 0.0
    tkval = tqdm(valloader, total=int(len(valloader)))
    for step, batch in enumerate(tkval):
        inputs = batch[0].to(device, dtype=torch.float)
        labels = batch[1].to(device).long()
        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        valpreds .append(outputs)
        running_loss += loss.item()
        tkval.set_postfix(valid_loss=(running_loss / (step+1)))
    preds = torch.cat(valpreds).argmax(1).detach().cpu().numpy()
    print(f'Valid accuracy {(valdf.label.values == preds).mean():.4f}')

In [13]:
# # With mix up
# since = time.time()
# for epoch in range(num_epochs):
#     print('Epoch {}/{}'.format(epoch, num_epochs - 1))
# #     scheduler.step()
#     model.train()
#     running_loss = 0.0
#     tk0 = tqdm(trnloader, total=int(len(trnloader)))
#     for step, batch in enumerate(tk0):
#         inputs = batch[0].to(device, dtype=torch.float)
#         labels = batch[1].to(device).long()
        
#         # mix up
#         beta = 1  
#         lam = np.random.beta(beta, beta)
#         rand_index = torch.randperm(inputs.size()[0]).to(device) # make an index which reorders the batch

#         labels_a = labels   
#         labels_b = labels[rand_index]  
        
#         inputs_mixed = lam * inputs + (1 - lam) * inputs[rand_index]    # Mixing inputs array with a randomized version of inputs array to produce the mix  

#         optimizer.zero_grad()
        
#         outputs = model(inputs_mixed)
#         loss = mixup_loss(criterion, outputs, labels_a, labels_b, lam)
        
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         tk0.set_postfix(train_loss=(running_loss / (step+1)))
    
#     scheduler.step(epoch=epoch)    
    
#     valpreds = []
#     model.eval()
#     running_loss = 0.0
#     tkval = tqdm(valloader, total=int(len(valloader)))
#     for step, batch in enumerate(tkval):
#         inputs = batch[0].to(device, dtype=torch.float)
#         labels = batch[1].to(device).long()
#         with torch.no_grad():
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#         valpreds .append(outputs)
#         running_loss += loss.item()
#         tkval.set_postfix(valid_loss=(running_loss / (step+1)))
#     preds = torch.cat(valpreds).argmax(1).detach().cpu().numpy()
#     print(f'Valid accuracy {(valdf.label.values == preds).mean():.4f}')

In [14]:
# Submit 
tstpreds = []
tktst = tqdm(tstloader, total=int(len(tstloader)))
for step, batch in enumerate(tktst):
    inputs = batch[0].to(device, dtype=torch.float)
    with torch.no_grad():
        outputs = model(inputs)
        tstpreds.append(outputs)
predicted_labels = torch.cat(tstpreds).argmax(1).detach().cpu().numpy()
tstdf['label'] = predicted_labels

In [15]:
tstdf.filename = tstdf.filename.str.replace('test/test/', 'test/')
tstdf.to_csv('submission.csv', index = False)

In [16]:
tstdf.head()